In [4]:
from py2neo import Graph, Node, Relationship, Path # import or necessary classes

**Note**: to be able to connect successfully you will need to set option `dbms.security.auth_enabled` to `false` in `/etc/neo4j/neo4j-server.conf`.  You will need to edit this as a super user.  This works for Cassandra version 3.x, in version 2 this file is called `neo4j.conf`.

In [5]:
graph = Graph("http://localhost:7474/db/data/") # namespace for our db (neo4j stores everything there)
graph.delete_all()

In [6]:
# the data
jp = Node("Person", name="JP", email="jean.p.ebejer@um.edu.mt")
kenneth = Node("Person", name="Kenneth")
charlot = Node("Person", name="Charlot")
isaac = Node("Person", name="Isaac")
jp_lect_kenneth = Relationship(jp, "LECTURES", kenneth)
jp_lect_charlot = Relationship(jp, "LECTURES", charlot)
charlot_friend_isaac = Relationship(charlot, "FRIEND", isaac)
charlot_friend_kenneth = Relationship(charlot, "FRIEND", kenneth, until="Mar 2016")

In [7]:
# create the graph
graph.create(jp)
graph.create(kenneth)
graph.create(charlot)
graph.create(isaac)
graph.create(jp_lect_kenneth)
graph.create(jp_lect_charlot)
graph.create(charlot_friend_isaac)
graph.create(charlot_friend_kenneth)

You can find a really useful tool at [http://localhost:7474/browser/]

In [8]:
# Example of the cypher language
for record in graph.run("MATCH (p:Person) RETURN p.name AS name"):
    print(record['name'])

JP
Kenneth
Charlot
Isaac


In [9]:
dict(jp) # props of a node

{u'email': u'jean.p.ebejer@um.edu.mt', u'name': u'JP'}

In [10]:
graph.relationship_types # edge types

frozenset({u'FRIEND', u'LECTURES'})

In [11]:
# Note:  If using cassandra 2, you should use graph.cypher.execute (which is deprecated in v3)
# More details at: http://py2neo.org/v3/database.html

cur = graph.run("START n=node(*) return n") # return all nodes
for record in cur:
    print(record['n']['name'])

JP
Kenneth
Charlot
Isaac


In [12]:
# who does JP lecture ?
for n in graph.match(start_node=jp, rel_type="LECTURES"): 
    print n
    print(n.end_node()['name'])

(jp)-[:LECTURES]->(charlot)
Charlot
(jp)-[:LECTURES]->(kenneth)
Kenneth


In [13]:
# Isaac and Kenneth are not directly joined by an edge, so no results found
for n in graph.match(start_node=isaac, end_node=kenneth, bidirectional=True):
    print(n)

In [14]:
# from Kenneth traverse all FRIEND paths (bidirectional)
# cycles are considered once
n = graph.run('MATCH (Person { name:"Kenneth" })-[:FRIEND*1..]-(x) RETURN x')
for c in n:
    print c

(u'x': (charlot:Person {name:"Charlot"}))
(u'x': (isaac:Person {name:"Isaac"}))
